In [ ]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 12.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=aa50b7ee2d2f4b70507f227b67dcee3d9f3dd1a3ca29b6de43951e83cb81a8ca
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 12.5 MB/s 
     |████████████████████████████████| 4.0 MB 12.3 MB/s 
     |████████████████████████████████| 596 kB 5.6 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 47.8 MB/s 
     |████████████████████████████████| 6.6 MB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [ ]:
df['text'] = df['text'].astype('str') 

In [ ]:
#df_sample = df.sample(n=50000, random_state=2)

In [ ]:
#df_sample

In [ ]:
docs = list(df.loc[:, "text"].values)

In [ ]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [ ]:
178255 * 7

1247785

In [ ]:
docs = docs[1069530:1247785]

In [ ]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [3:26:45<00:00, 14.37it/s]


In [ ]:
df = df[1069530:1247785]

In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [ ]:
df

,text,Label,Negative,Neutral,Positive
1069530,India has avoided publicly denouncing Russia f...,1,0.062646,0.925260,0.012094
1069531,⚡️Video of the shelling of the TV tower.\n#Rus...,1,0.332644,0.659257,0.008099
1069532,"We hope Russia obeys rules of engagement, says...",1,0.080995,0.896394,0.022611
1069533,@anonukr You guys should stop pushing Ukrainia...,0,0.934140,0.063047,0.002813
1069534,Follow @m_suchkov for Russia/Ukraine updates.,1,0.006250,0.975841,0.017909
...,...,...,...,...,...
1247780,"Ukraine invaded, Russia under martial law.\n\n...",1,0.327247,0.657285,0.015468
1247781,Guten Tag from Germany where I’m helping out t...,1,0.022092,0.844269,0.133639
1247782,Business Insider - Bitcoin's strength is being...,1,0.007319,0.665347,0.327333
1247783,@Niyiafrika02 @Ukraine @Russia Our politicians...,1,0.232208,0.597477,0.170315


In [ ]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p7.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p7.csv "drive/My Drive/Colab Notebooks"

Mounted at drive
